### Image Colorization with OpenVino

This notebook demonstrates how to colorize images with OpenVINO using the Colorization model [colorization-v2](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/colorization-v2/README.md) from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md).



The idea here is given a grayscale image as input, the model hallucinates a plausible, vibrant & realistic colorized version of the image.

## Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from openvino.runtime import Core

## Download the model



In [1]:
!omz_downloader --name colorization-v2

################|| Downloading colorization-v2 ||################

========== Downloading model/public/colorization-v2/ckpt/colorization-v2-eccv16.pth
... 100%, 125953 KB, 7647 KB/s, 16 seconds passed

========== Downloading model/public/colorization-v2/model/__init__.py
... 100%, 0 KB, 93 KB/s, 0 seconds passed

========== Downloading model/public/colorization-v2/model/base_color.py
... 100%, 0 KB, 1414 KB/s, 0 seconds passed

========== Downloading model/public/colorization-v2/model/eccv16.py
... 100%, 4 KB, 15905 KB/s, 0 seconds passed

========== Replacing text in model/public/colorization-v2/model/__init__.py
========== Replacing text in model/public/colorization-v2/model/__init__.py
========== Replacing text in model/public/colorization-v2/model/eccv16.py



In [4]:
!omz_converter --name colorization-v2

========== Converting colorization-v2 to ONNX
Conversion to ONNX command: /home/art3mis/miniconda3/envs/openvino_env/bin/python -- /home/art3mis/miniconda3/envs/openvino_env/lib/python3.7/site-packages/openvino/model_zoo/internal_scripts/pytorch_to_onnx.py --model-path=model/public/colorization-v2 --model-name=ECCVGenerator --weights=model/public/colorization-v2/ckpt/colorization-v2-eccv16.pth --import-module=model --input-shape=1,1,256,256 --output-file=model/public/colorization-v2/colorization-v2-eccv16.onnx --input-names=data_l --output-names=color_ab

/home/art3mis/miniconda3/envs/openvino_env/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
ONN

In [1]:
import cv2
import numpy as np
from openvino.runtime import Core

/home/art3mis/miniconda3/envs/openvino_env/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


In [2]:
ie = Core()
model = ie.read_model(model="public/colorization-v2/FP16/colorization-v2.xml")
compiled_model = ie.compile_model(model=model, device_name="CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

In [3]:
N, C, H, W = list(input_layer.shape)

In [4]:
image = cv2.imread("test_images/test.jpg")
if image.shape[2] > 1:
    frame = cv2.cvtColor(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)
else:
    frame = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

In [8]:
img_rgb = frame.astype(np.float32) / 255
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
img_l_rs = cv2.resize(img_lab.copy(), (W, H))[:, :, 0]
inputs = {}
inputs['data_l'] = np.expand_dims(img_l_rs, axis=[0, 1])

In [9]:
infer_request = compiled_model.create_infer_request()

In [10]:
res = infer_request.infer(inputs)[output_layer]

In [14]:
update_res = np.squeeze(res)
out = update_res.transpose((1, 2, 0))
out = cv2.resize(out, (224, 224))
img_lab_out = np.concatenate((img_lab[:, :, 0][:, :, np.newaxis], out), axis=2)
img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)
original_image = cv2.resize(image,(224, 224))
grayscale_image = cv2.resize(frame, (224, 224))
colorize_image = (cv2.resize(img_bgr_out, (224, 224)) * 255).astype(np.uint8)
lab_image = cv2.resize(img_lab_out, (224, 224)).astype(np.uint8)

In [15]:
cv2.imwrite("Colorized.png", colorize_image)

True